#### **EEG ADHD Classification: CNN-LSTM with Bayesian Optimization**

This notebook implements a hybrid CNN-LSTM deep learning model for ADHD classification using EEG data. The model combines convolutional neural networks for spatial feature extraction with LSTM networks for temporal pattern recognition. Hyperparameter optimization is performed using Tree-structured Parzen Estimator (TPE) algorithm via Hyperopt, with an iterative convergence-based approach to ensure robust hyperparameter selection.

#### **Key Features**
- **Dual-stream architecture**: Combines raw EEG spatial-temporal features with engineered frequency band powers
- **Iterative Bayesian Optimization**: Runs multiple BO searches until standard deviation of results converges
- **Reproducible experiments**: Seed management for consistent results across runs
- **Comprehensive evaluation**: Multiple cross-validation strategies including Leave-One-Subject-Out
- **Early stopping**: Prevents overfitting with configurable patience parameter

#### **Table of Contents**

1. [First Imports](#first-imports) - Essential libraries for data processing, ML, and deep learning
2. [Read the Processed Dataset](#read-the-processed-dataset) - Load preprocessed EEG data with frequency features
3. [Group the Data](#group-the-data) - Reshape tabular data into 3D tensors for CNN-LSTM
4. [Dataset Loading](#dataset-loading) - Custom PyTorch Dataset with dual-stream architecture
5. [Model Creation](#model-creation) - Hybrid CNN-LSTM model with fusion and classification heads
6. [First Model Training](#first-model-training) - Baseline model training with default hyperparameters
7. [Helper Functions](#helper-functions) - Utilities for model creation, evaluation, and data loading
8. [Search Space Definition](#search-space-definition) - Hyperparameter search space for Bayesian Optimization
9. [Search Objective Definition](#search-objective-definition) - Objective function with train/test split or k-fold CV
10. [Hyperparameter Search](#hyperparameter-search) - TPE search with configurable iterations and seeding
11. [Results Visualization](#results-visualization) - Iterative BO with convergence tracking and analysis
    - Convergence-based optimization with stability criteria
    - Summary visualization of all BO runs
    - Best parameter extraction and final model training
12. [Cross-Validation Experiments](#cross-validation-experiments) - Window-based k-fold and LOSOCV evaluation

---

#### **First Imports**

Import essential libraries for data manipulation, machine learning, and deep learning:
- **NumPy/Pandas**: Data processing and manipulation
- **Scikit-learn**: Train/test splitting, metrics, and label encoding
- **PyTorch**: Deep learning framework for building and training neural networks
- **Torch Optimizers**: Adam, RMSprop, and SGD for model optimization

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GroupShuffleSplit

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, RMSprop, SGD

#### **Read the Processed Dataset**

Load the preprocessed and clustered ADHD EEG dataset. This dataset contains:
- **Frequency domain features**: Power spectral density values across different frequency bands
- **Window segments**: Temporal windows extracted from continuous EEG recordings
- **Electrode channels**: Data from 19 EEG electrodes plus 7 band power features
- **Subject IDs**: For cross-validation and subject-independent testing
- **Class labels**: ADHD diagnostic categories

In [ ]:
df = pd.read_csv("./processed_clustered_adhdata.csv")

frequency_count = len(df['Frequency'].unique())
window_count = len(df['Window'].unique())
numeric_df = df.drop(['ID', 'Window'], axis=1)

display(df.head())

#### **Group the Data**

Transform the 2D tabular data into 3D tensors suitable for CNN-LSTM processing:

**Reshaping Process:**
1. Group rows by window number to reconstruct temporal segments
2. Create shape: `(WINDOW_COUNT, FREQUENCY_PER_WINDOW, ELECTRODES)`
3. Separate features (X) from labels (y)
4. Add channel dimension for CNN compatibility: `(N, freq, electrodes, 1)`

**Train/Test Split:**
- 80% training, 20% testing
- Stratified sampling to maintain class distribution
- Ensures balanced representation of all diagnostic categories

In [ ]:
# shape: (windows, frequencies, electrodes)
full_ndarray = numeric_df.values.reshape((window_count, frequency_count, numeric_df.shape[1]))

X = full_ndarray[:, :, 2:]     # drop ID/Class columns
y = full_ndarray[:, 0, 0]      # class label is repeated across freq rows

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Add channel dimension (N, 1, freq, electrodes)
X_train = X_train[..., np.newaxis]   # (N, freq, electrodes, 1)
X_test  = X_test[...,  np.newaxis]

print(X_train.shape)
print("Train shape:", X_train.shape)  # (N, freq, electrodes, 1)

#### **Dataset Loading**

Custom PyTorch Dataset class for EEG data with dual-stream architecture:

**Features:**
- **EEG Raw Features**: Spatial-temporal patterns from 19 electrodes (1, 77, 19)
- **Band Power Features**: Pre-computed frequency band powers (7 features)
- **Data Augmentation Ready**: Supports future augmentation strategies
- **Batch Processing**: Efficient DataLoader integration

**Architecture Rationale:**
The dual-stream approach allows the model to learn both:
1. Fine-grained spatial-temporal patterns via CNN-LSTM
2. Engineered frequency domain features via dense layers

In [11]:
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2)
        # -> (N, 1, freq, electrodes)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]          # (1, 77, 26)

        x_eeg  = x[:, :, :19]    # (1, 77, 19)
        x_band = x[0, 0, 19:]    # (7,)

        return x_eeg, x_band, self.y[idx]

train_ds = EEGDataset(X_train, y_train)
test_ds  = EEGDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

#### **Model Creation**

Hybrid CNN-LSTM architecture with dual-stream feature fusion:

**CNN Branch (Spatial Feature Extraction):**
- Conv2D layers with configurable kernels and sizes
- Average pooling for downsampling
- Dropout for regularization
- Dense layer before LSTM for dimensionality reduction

**LSTM Branch (Temporal Pattern Recognition):**
- Multi-layer LSTM for sequence modeling
- Configurable hidden size and layer depth
- Dropout between LSTM layers
- Final timestep aggregation

**Band Power Branch:**
- Dense layers for frequency domain features
- ReLU activation and dropout

**Fusion & Classification:**
- Concatenate CNN-LSTM output with band power features
- Two-layer classification head
- Supports multi-class ADHD categorization

**Training Features:**
- Early stopping with patience parameter
- Best model checkpoint restoration
- Training/validation history tracking
- Overfitting detection capabilities

In [12]:
import torch
import torch.nn as nn
import numpy as np

class EEGCNNLSTM(nn.Module):
    def __init__(self, num_band_features=7, num_classes=4,
                 cnn_kernels_1=32,
                 cnn_kernel_size_1=3,
                 cnn_kernels_2=32,
                 cnn_kernel_size_2=3,
                 cnn_dropout=0.3,
                 cnn_dense=16,
                 lstm_hidden_size=32,
                 lstm_layers=4,
                 lstm_dense=64,
                 dropout=0.3):
        super().__init__()
        
        pad1 = cnn_kernel_size_1 // 2
        self.conv1   = nn.Conv2d(1, int(cnn_kernels_1), kernel_size=cnn_kernel_size_1, padding=pad1)
        self.pool1 = nn.AvgPool2d(2)
        
        pad2 = cnn_kernel_size_2 // 2
        self.conv2 = nn.Conv2d(int(cnn_kernels_1), int(cnn_kernels_2), kernel_size=cnn_kernel_size_2, padding=pad2)
        self.cnn_dropout = nn.Dropout(cnn_dropout)

        # Compute flatten size dynamically
        with torch.no_grad():
            dummy = torch.zeros(1, 1, X_train.shape[1], 19)
            out = self._forward_cnn(dummy)   # [B, C, H, W]
            b, c, h, w = out.shape
            self.seq_len = h                      # sequence length (rows)
            self.cnn_feat_dim = c * w             # CNN features per timestep

        # Dense layer BEFORE LSTM
        self.cnn_dense = nn.Linear(self.cnn_feat_dim, int(cnn_dense))

        # Two stacked LSTM layers
        self.lstm = nn.LSTM(
            input_size=int(cnn_dense),
            hidden_size=int(lstm_hidden_size),
            num_layers=int(lstm_layers),
            batch_first=True,
            dropout=dropout if lstm_layers > 1 else 0.0
        )
        # self.lstm_dense = nn.Linear(int(lstm_hidden_size), int(lstm_dense))
        self.band_fc = nn.Sequential(
            nn.Linear(num_band_features, 32),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        
        # final classifier (match your original final style: dropout + linear)
        self.classifier = nn.Sequential(
            nn.Linear(int(lstm_hidden_size) + 32, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def _forward_cnn(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.cnn_dropout(x)
        return x

    def forward(self, x_eeg, x_band):
        # 1️⃣ CNN feature extraction
        x = self._forward_cnn(x_eeg)             # [B, C, H, W]

        # 2️⃣ Prepare sequence for LSTM
        x = x.permute(0, 2, 1, 3)                 # [B, H, C, W]
        x = x.contiguous().view(x.size(0), x.size(1), -1)  # [B, H, C*W]

        # 3️⃣ Dense layer for each timestep
        x = F.relu(self.cnn_dense(x))     # [B, H, dense_size]

        # 4️⃣ Two-layer LSTM
        lstm_out, _ = self.lstm(x)                # [B, H, hidden_size]

        # 5️⃣ Use last time step (or mean/attention if preferred)
        eeg_feat = lstm_out[:, -1, :]
        # eeg_feat = lstm_out.mean(dim=1)                    # [B, hidden_size]
        # eeg_feat = self.lstm_dense(eeg_feat)

        # --- Band features ---
        band_feat = self.band_fc(x_band)        # [B, 32]

        # --- Fusion ---
        fused = torch.cat([eeg_feat, band_feat], dim=1)

        # 6️⃣ Fully connected head
        x = self.classifier(fused)

        return x

    def fit(self, train_loader, test_loader, epochs, criterion, optimizer, device, patience=100):
        best_val_loss = float('inf')
        no_improve = 0

        train_losses, train_accs = [], []
        val_losses, val_accs     = [], []

        best_state = None
        for epoch in range(epochs):
            # --- Train ---
            self.train()
            train_loss, train_correct, train_total = 0.0, 0, 0
            for xb_eeg, xb_band, yb in train_loader:
                xb_eeg = xb_eeg.to(device)
                xb_band = xb_band.to(device)
                yb = yb.to(device)
                optimizer.zero_grad()
                out = self(xb_eeg, xb_band)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * xb_eeg.size(0)
                train_correct += (out.argmax(1) == yb).sum().item()
                train_total += yb.size(0)

            train_loss /= train_total
            train_acc  = train_correct / train_total
            train_losses.append(train_loss)
            train_accs.append(train_acc)

            # --- Validate ---
            self.eval()
            val_loss, val_correct, val_total = 0.0, 0, 0
            with torch.no_grad():
                for xb_eeg, xb_band, yb in test_loader:
                    xb_eeg = xb_eeg.to(device)
                    xb_band = xb_band.to(device)
                    yb = yb.to(device)
                    out = self(xb_eeg, xb_band)
                    loss = criterion(out, yb)
                    val_loss += loss.item() * xb_eeg.size(0)
                    val_correct += (out.argmax(1) == yb).sum().item()
                    val_total += yb.size(0)

            val_loss /= val_total
            val_acc  = val_correct / val_total
            val_losses.append(val_loss)
            val_accs.append(val_acc)

            print(f"Epoch {epoch+1:03d} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

            # if val_loss - train_loss > 0.2:
            #     print("Overfitting detected.")
            #     break

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = self.state_dict()
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print("Early stopping triggered.")
                    break

        if best_state is not None:
            self.load_state_dict(best_state)
        return {
            "train_accs": np.array(train_accs),
            "train_losses": np.array(train_losses),
            "val_accs":   np.array(val_accs),
            "val_losses": np.array(val_losses)
        }

#### **First Model Training**

Initial baseline model training with default hyperparameters:

**Purpose:**
- Establish baseline performance metrics
- Verify model architecture and data pipeline
- Identify potential issues before hyperparameter optimization

**Training Configuration:**
- 60 epochs with early stopping
- Adam optimizer with L2 regularization
- Cross-entropy loss for multi-class classification
- Automatic GPU detection and utilization

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EEGCNNLSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

history = model.fit(train_loader, test_loader, epochs=60, criterion=criterion,
                    optimizer=optimizer, device=device)

In [ ]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb_eeg, xb_band, yb in test_loader:
        xb_eeg = xb_eeg.to(device)
        xb_band = xb_band.to(device)
        preds = model(xb_eeg, xb_band).argmax(1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(yb.numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

print("\nTest Accuracy:", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds))
print(confusion_matrix(all_labels, all_preds))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_accs'], label='Training Accuracy')
plt.plot(history['val_accs'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_losses'], label='Training Loss')
plt.plot(history['val_losses'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

#### **Helper Functions**

Utility functions for hyperparameter optimization and model evaluation:

**Core Functions:**
- `batched()`: Iterator utility for creating batches (for cross-validation folds)
- `get_timestamp()`: Logging timestamp generation
- `get_model()`: Model instantiation with custom hyperparameters
- `get_validation()`: Comprehensive model evaluation with metrics
- `get_dataset()`: Dynamic dataset creation with configurable batch sizes

**Evaluation Metrics:**
- Accuracy score
- Classification report (precision, recall, F1-score)
- Confusion matrix
- Cross-validation support

These functions enable efficient experimentation and reproducible results across different hyperparameter configurations.

In [9]:
import itertools

def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 2) → AB CD EF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(itertools.islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import time
import datetime

def get_timestamp():
    ts = time.time()
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

def get_model(params):
    model = EEGCNNLSTM(
        cnn_kernels_1=params['cnn_kernels_1'],
        cnn_kernel_size_1=params['cnn_kernel_size_1'],
        cnn_kernels_2=params['cnn_kernels_2'],
        cnn_dropout=float(params['cnn_dropout']),
        cnn_dense=params['cnn_dense'],
        lstm_hidden_size=params['lstm_hidden_size'],
        lstm_layers=params['lstm_layers'],
        lstm_dense=params['lstm_dense'],
        dropout=float(params['cnn_dropout']),  # use cnn_dropout as a simple shared dropout param
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    if params['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=1e-4)
    elif params['optimizer'] == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=params['learning_rate'], weight_decay=1e-4)
    else:
        optimizer = optim.SGD(model.parameters(), lr=params['learning_rate'], momentum=0.9, weight_decay=1e-4)

    return model, criterion, optimizer

def get_validation(model, data_loader, device, matrix=True):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb_eeg, xb_band, yb in data_loader:
            xb_eeg = xb_eeg.to(device)
            xb_band = xb_band.to(device)
            preds = model(xb_eeg, xb_band).argmax(1).cpu().numpy()
            all_preds.append(preds)
            all_labels.append(yb.numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds) if matrix else None

    return acc, report, conf_matrix

def get_dataset(df, is_train=False, batch_size=36):
    frequency_count = len(df['Frequency'].unique())
    window_count = len(df['Window'].unique())
    numeric_df = df.drop(['ID', 'Window'], axis=1)

    # shape: (windows, freqs, features)
    full_ndarray = numeric_df.values.reshape((window_count, frequency_count, numeric_df.shape[1]))

    X = full_ndarray[:, :, 2:]     # drop ID/Class columns
    y = full_ndarray[:, 0, 0]      # class label is repeated across freq rows

    # Add channel dimension (N, 1, freq, electrodes)
    X = X[..., np.newaxis]          # (N, freq, electrodes, 1)

    print(X.shape)

    return DataLoader(EEGDataset(X, y), batch_size=batch_size, shuffle=is_train)

#### **Search Space Definition**

Hyperparameter search space for Bayesian Optimization using Hyperopt:

**CNN Architecture:**
- `cnn_kernels_1/2`: Number of convolutional filters [16, 32, 48, 64, 96]
- `cnn_kernel_size_1/2`: Kernel dimensions [3x3, 5x5]
- `cnn_dropout`: Regularization rate [0.0 - 0.7]
- `cnn_dense`: Dense layer size before LSTM [32, 64, 128, 256]

**LSTM Architecture:**
- `lstm_hidden_size`: Hidden state dimensions [32, 64, 96, 128]
- `lstm_layers`: Number of stacked LSTM layers [1-6]
- `lstm_dense`: Dense layer size after LSTM [32, 64, 128, 256]

**Training Configuration:**
- `learning_rate`: Log-uniform distribution [1e-5, 1e-2]
- `optimizer`: Choice of Adam, RMSprop, or SGD
- `batch_size`: Samples per batch [32, 36, 48, 64, 80, 96]

**Optimization Strategy:**
Tree-structured Parzen Estimator (TPE) algorithm balances exploration and exploitation to efficiently search the hyperparameter space.

In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK

# -------------------------
# Hyperopt search space
# -------------------------
space = {
    'cnn_kernels_1'    : hp.choice('cnn_kernels_1', [16, 32, 48, 64]),
    'cnn_kernel_size_1': hp.choice('cnn_kernel_size_1', [3, 5]),
    'cnn_kernels_2'    : hp.choice('cnn_kernels_2', [16, 32, 64, 96]),
    'cnn_kernel_size_2': hp.choice('cnn_kernel_size_2', [3, 5]),
    'cnn_dropout'      : hp.uniform('cnn_dropout', 0.0, 0.7),
    'cnn_dense'        : hp.choice('cnn_dense', [32, 64, 128, 256]),
    'lstm_hidden_size' : hp.choice('lstm_hidden_size', [32, 64, 96, 128]),
    'lstm_layers'      : hp.choice('lstm_layers', [1, 2, 3, 4, 5, 6]),
    'lstm_dense'       : hp.choice('lstm_dense', [32, 64, 128, 256]),
    'learning_rate'    : hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-2)),
    'optimizer'        : hp.choice('optimizer', ['adam', 'rmsprop', 'sgd']),
    'batch_size'       : hp.choice('batch_size', [32, 36, 48, 64, 80, 96])
}

#### **Search Objective Definition**

Objective function for Hyperopt optimization with two strategies:

### Strategy 1: K-Fold Cross-Validation (Commented)
**Approach:**
- Subject-stratified K-fold cross-validation
- Cyclic fold generation for balanced distribution
- Score calculation: `mean_loss + variance_of_tail_losses`
- More robust but computationally expensive

**Metrics:**
- Mean validation loss across all folds
- Variance of final validation losses (stability measure)
- Combined score penalizes both high loss and instability

### Strategy 2: Single Train/Test Split (Active)
**Approach:**
- Uses predefined train/test split
- Faster iteration for initial hyperparameter search
- Early stopping with patience=10

**Return Value:**
- Minimizes best validation loss
- Includes training history for analysis
- STATUS_OK for successful trials

**Note:** The objective function is called by Hyperopt's `fmin()` and should return a dictionary with 'loss' and 'status' keys.

In [12]:
import itertools

# -------------------------
# k-Fold CV for Hyperopt
# -------------------------
def objective(params):
    print("Trial params:", params)

    criterion = nn.CrossEntropyLoss()
    unique_subjects = df['ID'].unique()
    losses = []
    variances = []
    batch_size = params['batch_size']

    K_FOLDS = 5
    fold_size = len(unique_subjects) // K_FOLDS

    cyclic = itertools.cycle(unique_subjects)
    batched_cyclic = batched(cyclic, n=fold_size)
    folds = itertools.islice(batched_cyclic, K_FOLDS)

    for i, fold in enumerate(folds):
        print(f"Starting fold {i + 1}/{K_FOLDS}")

        train_df = df[~df['ID'].isin(fold)]
        test_df  = df[df['ID'].isin(fold)]

        print(train_df.shape, test_df.shape)

        train_loader = get_dataset(train_df, batch_size=batch_size)
        test_loader  = get_dataset(test_df, batch_size=batch_size)
        model, criterion, optimizer = get_model(params)

        # Train with modest epochs; early stopping inside fit handles rest
        history = model.fit(
            train_loader=train_loader,
            test_loader=test_loader,
            epochs=60,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            patience=15
        )

        acc, *_ = get_validation(model, test_loader, device)
        loss = history['val_losses']
        mean_loss = np.min(loss)
        losses.append(mean_loss)

        last_5_or_less = history["val_losses"]
        last_5_or_less = last_5_or_less[-min(len(last_5_or_less), 5):]
        variance = np.var(last_5_or_less) if len(last_5_or_less) > 1 else 1
        variances.append(variance)

        print(f"Fold {i + 1} Accuracy:", acc)

    loss = np.mean(losses)
    tail_variance = np.var(variances)
    print(variances)
    score = loss + tail_variance

    print(f"k-Fold CV Mean Loss: {loss:.4f} ± {np.std(losses):.4f}")
    print(f"k-Fold CV Tail Variance: {tail_variance:.4f}")

    # Hyperopt minimizes -> return negative accuracy
    return {'loss': score, 'status': STATUS_OK, 'attachments': {'history': history}}


In [18]:
# -------------------------
# Single Objective for Hyperopt
# -------------------------
def objective(params):
    print("Trial params:", params)

    # build dataloaders from the existing train_ds/test_ds in this session
    train_loader = DataLoader(train_ds, batch_size=params['batch_size'], shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=params['batch_size'], shuffle=False)

    # create model (note we pass dropout into lstm dropout and cnn dropout)
    model, criterion, optimizer = get_model(params)

    # Train with modest epochs; early stopping inside fit handles rest
    history = model.fit(
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=60,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        patience=10
    )

    best_val_loss = float(np.min(history['val_losses'])) if len(history['val_losses']) > 0 else 0.0

    # Hyperopt minimizes -> return negative accuracy
    return {'loss': best_val_loss, 'status': STATUS_OK, 'attachments': {'history': history, 'best_val_loss': best_val_loss}}


#### **Hyperparameter Search**

Execute Tree-structured Parzen Estimator (TPE) search for optimal hyperparameters:

**Search Configuration:**
- `max_evals`: Number of trials (default: 30, increase for thorough search)
- Algorithm: TPE (adaptive Bayesian optimization)
- Tracks all trials in Hyperopt Trials object

**Process:**
1. Initialize trials tracking object
2. Run TPE algorithm over search space
3. Convert raw indices to interpretable values
4. Save best parameters to JSON

**Output:**
- **Best hyperparameters**: Both raw indices and interpreted values
- **Search duration**: Total optimization time
- **Parameters JSON**: Serialized configuration for model reproduction

**Multiple Runs:**
The second cell executes 10 independent search runs to:
- Assess hyperparameter sensitivity
- Identify robust configurations
- Build ensemble of candidate models

In [ ]:
from hyperopt import Trials, fmin

def hyperparameter_search(max_evals=30, seed=None):
    trials = Trials()
    
    # Create seeded random state for Hyperopt
    rstate = np.random.RandomState(seed) if seed is not None else None
    
    print("Starting TPE search...")
    if seed is not None:
        print(f"Using seed: {seed}")
    t0 = time.time()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,   # increase for more thorough search
        trials=trials,
        rstate=rstate  # CRITICAL: seed Hyperopt's random state
    )
    
    print("Best hyperparameters:", best)
    t1 = time.time()
    duration = t1 - t0
    print(f"TPE search finished in {duration:.2f} seconds")
    print("Best (raw indices):", best)
    
    # Convert choice indices back to values for readability:
    def choice_value(key, val):
        mapping = {
            'cnn_kernels_1': [16, 32, 48, 64],
            'cnn_kernel_size_1': [3, 5],
            'cnn_kernels_2': [16, 32, 64, 96],
            'cnn_kernel_size_2': [3, 5],
            'cnn_dense': [32, 64, 128, 256],
            'lstm_hidden_size': [32, 64, 96, 128],
            'lstm_layers': [1, 2, 3, 4, 5, 6],
            'lstm_dense': [32, 64, 128, 256],
            'optimizer': ['adam', 'rmsprop', 'sgd'],
            'batch_size': [32, 36, 48, 64, 80, 96]
        }
        return mapping[key][int(val)] if key in mapping else val
    
    readable = {k: choice_value(k, v) if k in ['cnn_kernels_1','cnn_kernel_size_1','cnn_kernels_2',
                                               'cnn_kernel_size_2', 'cnn_dense','lstm_hidden_size',
                                               'lstm_layers','lstm_dense','optimizer','batch_size'] else v
                for k,v in best.items()}
    print("Best (interpreted):", readable)

    params = dict(readable)
    params['cnn_kernels_1'] = int(params['cnn_kernels_1'])
    params['cnn_kernel_size_1'] = int(params['cnn_kernel_size_1'])
    params['cnn_kernels_2'] = int(params['cnn_kernels_2'])
    params['cnn_kernel_size_2'] = int(params['cnn_kernel_size_2'])
    params['cnn_dense'] = int(params['cnn_dense'])
    params['lstm_hidden_size'] = int(params['lstm_hidden_size'])
    params['lstm_layers'] = int(params['lstm_layers'])
    params['lstm_dense'] = int(params['lstm_dense'])
    params['batch_size'] = int(params['batch_size'])
    params['cnn_dropout'] = float(params['cnn_dropout'])
    params['dropout'] = float(params['cnn_dropout'])

    return trials, params

In [ ]:
trials, params = hyperparameter_search()
with open("best_parameters.json", "w+") as f:
    import json
    json.dump(params, f, indent=4)

In [ ]:
trial_results = []

for i in range(10):
    trials, params = hyperparameter_search()
    trial_results.append({"trials": trials, "params": params})

In [ ]:
import json
import numpy as np
from datetime import datetime


def to_json_safe(obj):
    """Recursively convert objects to JSON-serializable types."""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    if isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    if isinstance(obj, datetime):
        return obj.isoformat()
    if isinstance(obj, dict):
        return {k: to_json_safe(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_json_safe(v) for v in obj]
    return obj


def serialize_trial(trial):
    """Extract only JSON-safe and meaningful parts of a Hyperopt Trial."""
    print(trial.attachments)
    return {
        "attachments": to_json_safe(trial.attachments)
    }


# ===============================
# DROP-IN REPLACEMENT STARTS HERE
# ===============================

json_results = []

for entry in trial_results:
    json_results.append({
        "params": to_json_safe(entry.get("params")),
        "trial": serialize_trial(entry.get("trials")),
    })

with open("all_trials.json", "w", encoding="utf-8") as f:
    json.dump(
        {"results": json_results},
        f,
        indent=4,
        ensure_ascii=False
    )

#### **Results Visualization**

Comprehensive hyperparameter optimization with iterative Bayesian Optimization and convergence tracking:

---

### **Iterative Convergence-Based Bayesian Optimization**

**Multi-Run Strategy:**
- Starts with 3 initial BO searches with different random seeds
- Runs 2 additional iterations and recalculates standard deviation
- Continues until std dev stabilizes (relative change < 0.1%)
- Tracks convergence to ensure robust hyperparameter selection

**Reproducibility:**
- Each BO run uses a unique sequential seed (base_seed + run_index)
- Seeds control PyTorch, NumPy, Python random states, and Hyperopt's TPE sampler
- Full seed tracking for experiment reproduction

**Stability Criteria:**
- Convergence: `|current_std - previous_std| / previous_std < 0.001`
- Reduces sensitivity to random initialization
- Ensures identified hyperparameters are stable across multiple searches

**Safety Measures:**
- Maximum limit of 20 total BO runs
- Prevents infinite loops while allowing thorough exploration

---

### **Comprehensive Visualization and Analysis**

**Performance Summary:**
- Table of all BO runs with seeds and validation losses
- Ranking of configurations by performance
- Statistical analysis: mean, std dev, min/max, range

**Convergence Plots:**
- Best validation loss across all runs (bar chart with annotations)
- Standard deviation evolution over iterations
- Percentage change annotations between measurements
- Color-coded convergence indicators (green when < 0.1% change)

**Visual Highlights:**
- Best performing run highlighted in green
- Mean and minimum loss reference lines
- Convergence threshold visualization

---

### **Best Parameters Extraction**

**Selection Process:**
- Systematically searches across all BO runs
- Identifies trial with lowest validation loss
- Extracts complete hyperparameter configuration

**Output:**
- Best run index and seed for reproducibility
- Complete hyperparameter dictionary
- Best validation loss achieved
- Saves to `best_parameters.json` for deployment

---

### **Final Model Training with Optimal Hyperparameters**

**Evaluation:**
- Trains model with best discovered hyperparameters
- Reports test set accuracy, classification report, confusion matrix
- Visualizes training history (accuracy and loss curves)
- Confirms model generalization performance

In [ ]:
import random
import numpy as np
import torch

def set_seed(seed):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def run_bo_with_seed(seed, max_evals=30):
    """Run Bayesian Optimization with a specific seed."""
    print(f"\n{'='*60}")
    print(f"Running BO with seed: {seed}")
    print(f"{'='*60}")
    
    set_seed(seed)
    # CRITICAL: Pass seed to hyperparameter_search to seed Hyperopt
    trials, params = hyperparameter_search(max_evals=max_evals, seed=seed)
    
    # Extract best validation loss from this run
    best_val_loss = float('inf')
    for trial in trials.trials:
        if trial['result']['status'] == 'ok':
            loss = trial['result']['loss']
            if loss < best_val_loss:
                best_val_loss = loss
    
    return {
        'seed': seed,
        'trials': trials,
        'params': params,
        'best_val_loss': best_val_loss
    }

# Initialize tracking variables
all_bo_runs = []
std_devs = []
run_counts = []  # Track number of runs for each std_dev calculation
base_seed = 42

print("Starting iterative Bayesian Optimization with convergence tracking...")
print(f"Convergence criterion: Standard deviation change < 0.1%\n")

# Initial 3 runs
print("Phase 1: Running initial 3 BO searches...")
for i in range(3):
    seed = base_seed + i
    result = run_bo_with_seed(seed, max_evals=30)
    all_bo_runs.append(result)
    
    print(f"\nRun {i+1}/3 completed:")
    print(f"  Seed: {seed}")
    print(f"  Best Val Loss: {result['best_val_loss']:.6f}")

# Calculate std dev only after all 3 initial runs
best_losses = [r['best_val_loss'] for r in all_bo_runs]
current_std = np.std(best_losses)
std_devs.append(current_std)
run_counts.append(len(all_bo_runs))

print(f"\nPhase 1 Summary:")
print(f"  Total Runs: {len(all_bo_runs)}")
print(f"  Std Dev: {current_std:.6f}")

# Iterative refinement
iteration = 2
converged = False
prev_std = std_devs[-1]

while not converged:
    print(f"\n{'='*60}")
    print(f"Phase {iteration}: Running 2 additional BO searches...")
    print(f"{'='*60}")
    
    # Run 2 more BO searches
    for i in range(2):
        seed = base_seed + len(all_bo_runs)
        result = run_bo_with_seed(seed, max_evals=30)
        all_bo_runs.append(result)
        
        print(f"\nRun {len(all_bo_runs)} completed:")
        print(f"  Seed: {seed}")
        print(f"  Best Val Loss: {result['best_val_loss']:.6f}")
    
    # Calculate new std dev after both runs complete
    best_losses = [r['best_val_loss'] for r in all_bo_runs]
    current_std = np.std(best_losses)
    std_devs.append(current_std)
    run_counts.append(len(all_bo_runs))
    
    # Check convergence
    std_change = abs(current_std - prev_std) / prev_std if prev_std > 0 else 1.0
    
    print(f"\n--- Convergence Check ---")
    print(f"Total Runs:       {len(all_bo_runs)}")
    print(f"Previous Std Dev: {prev_std:.6f}")
    print(f"Current Std Dev:  {current_std:.6f}")
    print(f"Relative Change:  {std_change*100:.4f}%")
    
    if std_change < 0.001:  # 0.1% threshold
        print(f"\n✓ Convergence achieved! Std dev change < 0.1%")
        converged = True
    else:
        print(f"\n→ Not converged yet, continuing...")
        prev_std = current_std
        iteration += 1
        
        # Safety limit
        if len(all_bo_runs) >= 20:
            print(f"\n⚠ Reached maximum of 20 runs, stopping.")
            converged = True

print(f"\n{'='*60}")
print(f"Optimization Complete!")
print(f"{'='*60}")
print(f"Total BO runs: {len(all_bo_runs)}")
print(f"Final Std Dev: {std_devs[-1]:.6f}")
print(f"Std Dev tracked at runs: {run_counts}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create summary table
summary_data = []
for i, run in enumerate(all_bo_runs):
    summary_data.append({
        'Run': i + 1,
        'Seed': run['seed'],
        'Best Val Loss': run['best_val_loss'],
        'Rank': 0  # Will be filled after sorting
    })

# Sort by best val loss and assign ranks
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Best Val Loss')
summary_df['Rank'] = range(1, len(summary_df) + 1)
summary_df = summary_df.sort_values('Run')  # Resort by run order

print("\n" + "="*70)
print("SUMMARY OF BAYESIAN OPTIMIZATION RUNS")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

# Calculate statistics
mean_loss = summary_df['Best Val Loss'].mean()
std_loss = summary_df['Best Val Loss'].std()
min_loss = summary_df['Best Val Loss'].min()
max_loss = summary_df['Best Val Loss'].max()

print(f"\nStatistics:")
print(f"  Mean Best Loss: {mean_loss:.6f}")
print(f"  Std Dev:        {std_loss:.6f}")
print(f"  Min Loss:       {min_loss:.6f}")
print(f"  Max Loss:       {max_loss:.6f}")
print(f"  Range:          {max_loss - min_loss:.6f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Best validation losses across runs
ax1 = axes[0]
runs = summary_df['Run'].values
losses = summary_df['Best Val Loss'].values
colors = ['#2ecc71' if loss == min_loss else '#3498db' for loss in losses]

bars = ax1.bar(runs, losses, color=colors, alpha=0.7, edgecolor='black', linewidth=1.2)
ax1.axhline(y=mean_loss, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_loss:.4f}')
ax1.axhline(y=min_loss, color='green', linestyle='--', linewidth=2, alpha=0.5, label=f'Best: {min_loss:.4f}')

ax1.set_xlabel('BO Run', fontsize=12, fontweight='bold')
ax1.set_ylabel('Best Validation Loss', fontsize=12, fontweight='bold')
ax1.set_title('Best Validation Loss per BO Run', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)
ax1.set_xticks(runs)

# Annotate the best run
best_run_idx = summary_df['Best Val Loss'].idxmin()
best_run = summary_df.loc[best_run_idx, 'Run']
best_loss = summary_df.loc[best_run_idx, 'Best Val Loss']
ax1.annotate(f'Best\n{best_loss:.4f}', 
             xy=(best_run, best_loss),
             xytext=(best_run, best_loss + (max_loss - min_loss) * 0.1),
             ha='center',
             fontsize=9,
             fontweight='bold',
             color='darkgreen',
             arrowprops=dict(arrowstyle='->', color='darkgreen', lw=1.5))

# Plot 2: Standard deviation convergence
ax2 = axes[1]

# Use the tracked run_counts for accurate x-axis
ax2.plot(run_counts, std_devs, marker='o', linewidth=2.5, markersize=8, 
         color='#e74c3c', markerfacecolor='white', markeredgewidth=2)
ax2.set_xlabel('Number of BO Runs', fontsize=12, fontweight='bold')
ax2.set_ylabel('Std Dev of Best Losses', fontsize=12, fontweight='bold')
ax2.set_title('Convergence of Standard Deviation', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add convergence threshold line
if len(std_devs) > 1:
    converged_std = std_devs[-1]
    ax2.axhline(y=converged_std, color='green', linestyle='--', linewidth=2, 
                alpha=0.5, label=f'Final: {converged_std:.4f}')
    ax2.legend(fontsize=10)

# Annotate percentage changes between consecutive measurements
for i in range(1, len(std_devs)):
    pct_change = abs(std_devs[i] - std_devs[i-1]) / std_devs[i-1] * 100 if std_devs[i-1] > 0 else 0
    mid_x = (run_counts[i] + run_counts[i-1]) / 2
    mid_y = (std_devs[i] + std_devs[i-1]) / 2
    
    # Color code: red if change > 0.1%, green if <= 0.1%
    box_color = 'lightgreen' if pct_change <= 0.1 else 'yellow'
    
    ax2.annotate(f'{pct_change:.2f}%', 
                xy=(mid_x, mid_y),
                fontsize=8,
                ha='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor=box_color, alpha=0.5, edgecolor='gray'))

plt.tight_layout()
plt.show()

print(f"\n✓ Visualization complete.")
print(f"\nConvergence History:")
for i, (runs, std) in enumerate(zip(run_counts, std_devs)):
    if i == 0:
        print(f"  After {runs} runs: Std Dev = {std:.6f}")
    else:
        prev_std = std_devs[i-1]
        pct_change = abs(std - prev_std) / prev_std * 100 if prev_std > 0 else 0
        status = "✓ Converged" if pct_change < 0.1 else "→ Continuing"
        print(f"  After {runs} runs: Std Dev = {std:.6f} (Change: {pct_change:.2f}%) {status}")

In [ ]:
import json

def extract_best_params_from_runs(all_runs):
    """Extract parameters from the best performing BO run."""
    best_overall_loss = float('inf')
    best_params = None
    best_run_index = -1
    best_seed = None
    
    for run_idx, run_data in enumerate(all_runs):
        if run_data['best_val_loss'] < best_overall_loss:
            best_overall_loss = run_data['best_val_loss']
            best_params = run_data['params']
            best_run_index = run_idx + 1
            best_seed = run_data['seed']
    
    if best_params:
        print(f"\n{'='*70}")
        print("BEST PERFORMING CONFIGURATION")
        print(f"{'='*70}")
        print(f"Run Index:        {best_run_index}/{len(all_runs)}")
        print(f"Seed:             {best_seed}")
        print(f"Best Val Loss:    {best_overall_loss:.6f}")
        print(f"\nHyperparameters:")
        print(json.dumps(best_params, indent=4))
        print(f"{'='*70}")
        
        # Save best parameters
        output_data = {
            'run_index': best_run_index,
            'seed': best_seed,
            'best_val_loss': float(best_overall_loss),
            'parameters': best_params
        }
        
        with open("best_parameters.json", "w") as f:
            json.dump(output_data, f, indent=4)
        
        print(f"\n✓ Best parameters saved to 'best_parameters.json'")
        
        return best_params
    else:
        print("Error: No valid parameters found.")
        return None

# Extract best parameters from all BO runs
best_params = extract_best_params_from_runs(all_bo_runs)

In [ ]:
# build dataloaders from the existing train_ds/test_ds in this session
train_loader = DataLoader(train_ds, batch_size=best_params['batch_size'], shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=best_params['batch_size'], shuffle=False)

# create model (note we pass dropout into lstm dropout and cnn dropout)
model, criterion, optimizer = get_model(best_params)

# Train with modest epochs; early stopping inside fit handles rest
history = model.fit(
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=60,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    patience=10
)
acc, report, matrix = get_validation(model, test_loader, device)

print("\nval Accuracy:", acc)
print(report)
print(matrix)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_accs'], label='Training Accuracy')
plt.plot(history['val_accs'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_losses'], label='Training Loss')
plt.plot(history['val_losses'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

#### **Cross-Validation Experiments**

Comprehensive model evaluation using multiple cross-validation strategies to assess generalization:

---

### **Experiment 1: K-Fold Window-Based Cross-Validation**

**Configuration:**
- K=11 folds
- Split by temporal window segments (not subjects)
- Random shuffle before folding
- Uses cyclic iterator for balanced fold distribution

**Purpose:**
- Assess model generalization to unseen temporal windows
- Tests ability to classify different time segments
- Faster than subject-based CV
- Subject data can appear in both train and test sets

**Characteristics:**
- May overestimate performance (data leakage from same subject)
- Good for assessing temporal pattern learning
- Less representative of real-world deployment

---

### **Experiment 2: Leave-One-Subject-Out Cross-Validation (LOSOCV)**

**Configuration:**
- Folds equal to number of unique subjects
- Each fold: one subject for testing, all others for training
- Windows from same subject grouped together
- No data leakage between train/test

**Purpose:**
- **Gold standard** for subject-independent evaluation
- Tests true generalization to new, unseen individuals
- Most realistic clinical deployment scenario
- Accounts for inter-subject variability

**Clinical Relevance:**
- Simulates real diagnostic workflow
- Model must generalize to patients not in training set
- Identifies subject-specific overfitting
- More conservative performance estimate

---

### **Evaluation Metrics**

Both experiments report:
- **Mean accuracy** across all folds
- **Standard deviation** (indicates model stability)
- **Per-fold accuracy** for detailed analysis

**Interpretation:**
- Lower std dev = more stable, reliable model
- Window-based: baseline generalization capability
- Subject-based: clinically relevant performance

---

### **Trade-offs Summary**

| Aspect | Window-Based K-Fold | Leave-One-Subject-Out |
|--------|---------------------|------------------------|
| **Speed** | Faster | Slower (more folds) |
| **Realism** | Less realistic | Highly realistic |
| **Performance** | Often higher | More conservative |
| **Data Leakage** | Possible | None |
| **Clinical Value** | Limited | High |
| **Use Case** | Quick validation | Final evaluation |

Both approaches validate model robustness from complementary perspectives and help identify different types of overfitting.

In [ ]:
import random

# map params to integers where needed
params = {'batch_size': 64, 'cnn_dense': 256, 'cnn_dropout': np.float64(0.3571401842400106), 'cnn_kernel_size_1': 5, 'cnn_kernel_size_2': 3, 'cnn_kernels_1': 32, 'cnn_kernels_2': 16, 'learning_rate': np.float64(1.8587640600578385e-05), 'lstm_dense': 64, 'lstm_hidden_size': 96, 'lstm_layers': 2, 'optimizer': 'sgd'}

criterion = nn.CrossEntropyLoss()
unique_subjects = list(df['Window'].unique())
accs = []
variances = []
batch_size = params['batch_size']

K_FOLDS = 11
fold_size = len(unique_subjects) // K_FOLDS

random.shuffle(unique_subjects)

cyclic = itertools.cycle(unique_subjects)
batched_cyclic = batched(cyclic, n=fold_size)
folds = itertools.islice(batched_cyclic, K_FOLDS)
    
for i, fold in enumerate(folds):
    print(f"Starting fold {i + 1}/{K_FOLDS}")

    train_df = df[~df['Window'].isin(fold)]
    test_df   = df[df['Window'].isin(fold)]

    train_loader = get_dataset(train_df, batch_size=batch_size)
    test_loader  = get_dataset(test_df, batch_size=batch_size)
    model, criterion, optimizer = get_model(params)

    # Train with modest epochs; early stopping inside fit handles rest
    history = model.fit(
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=100,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        patience=15
    )

    acc, *_ = get_validation(model, test_loader, device)
    accs.append(acc)

    print(f"Fold {i + 1} Accuracy:", acc)

print(f"k-Fold CV Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")

In [ ]:
import random

params = readable

unique_samples = list(df['Window'].unique())
subject_count = df['ID'].nunique()
accs = []
variances = []
batch_size = params['batch_size']

fold_size = len(unique_samples) // subject_count
random.shuffle(unique_samples)

cyclic = itertools.cycle(unique_samples)
batched_cyclic = batched(cyclic, n=fold_size)
folds = itertools.islice(batched_cyclic, subject_count)

for i, fold in enumerate(folds):
    print(f"[{get_timestamp()}] Starting fold {i + 1}/{subject_count}")

    train_df = df[~df['Window'].isin(fold)]
    test_df   = df[df['Window'].isin(fold)]

    train_loader = get_dataset(train_df, is_train=True, batch_size=batch_size)
    test_loader  = get_dataset(test_df, is_train=False, batch_size=batch_size)
    model, criterion, optimizer = get_model(params)

    # Train with modest epochs; early stopping inside fit handles rest
    history = model.fit(
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=60,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        patience=15
    )

    acc, *_ = get_validation(model, test_loader, device)
    accs.append(acc)

    print(f"Fold {i + 1} Accuracy:", acc)

print(f"LOSOCV Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")

In [ ]:

params = {'batch_size': 48, 'cnn_dense': 64, 'cnn_dropout': np.float64(0.5130373328759822), 'cnn_kernel_size_1': 5, 'cnn_kernel_size_2': 5, 'cnn_kernels_1': 16, 'cnn_kernels_2': 16, 'learning_rate': np.float64(0.000462968156587811), 'lstm_dense': 256, 'lstm_hidden_size': 128, 'lstm_layers': 1, 'optimizer': 'adam'}

df = pd.read_csv("./processed_clustered_adhdata.csv")

frequency_count = len(df['Frequency'].unique())
window_count = len(df['Window'].unique())
numeric_df = df.drop(['ID', 'Window'], axis=1)

display(df.head())

# shape: (windows, frequencies, electrodes)
full_ndarray = numeric_df.values.reshape((window_count, frequency_count, numeric_df.shape[1]))

X = full_ndarray[:, :, 2:]     # drop ID/Class columns
y = full_ndarray[:, 0, 0]      # class label is repeated across freq rows

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Add channel dimension (N, 1, freq, electrodes)
X_train = X_train[..., np.newaxis]   # (N, freq, electrodes, 1)
X_test  = X_test[...,  np.newaxis]

print(X_train.shape)

print("Train shape:", X_train.shape)  # (N, freq, electrodes, 1)
train_ds = EEGDataset(X_train, y_train)
test_ds  = EEGDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

unique_samples = list(df['Window'].unique())
subject_count = df['ID'].nunique()
accs = []
variances = []
batch_size = params['batch_size']

model, criterion, optimizer = get_model(params)

# Train with modest epochs; early stopping inside fit handles rest
history = model.fit(
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=60,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    patience=15
)

acc, *_ = get_validation(model, test_loader, device)
accs.append(acc)

print(f"Fold {i + 1} Accuracy:", acc)

print(f"LOSOCV Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")

In [ ]:
acc, report, matrix = get_validation(model, test_loader, device)

print("\nval Accuracy:", acc)
print(report)
print(matrix)